This notebook contains a detailed description of the data cleaning process and the exploratory data analysis (EDA) of the original data.<br><br> 
The **goal** of this analysis is to select or engineer features that can be used to predict fraud. <br> These features will be summarized in a new data frame that can be used for modelling. 

**Contents**

---

1. [**Import Libraries**](#-1.-Import-libraries)
2. [**Get Data**](#-2.-Get-data)
    *   data description 
    *   `df_client`: client data
    *   `df_invoice`: client invoice data 
3. [**Data Cleaning**](#-3.-Data-cleaning)
    *   [renaming](#renaming)
    *   check [missing values](#missing-values)
    *   check [duplicates](#duplicates)
    *   [data type conversions](#data-type-conversions)
4. [**Exploring Features**](#-4.-Exploratory-data-analysis-(EDA))

---

# 1. Import libraries 

In [1]:
import os
import sys
from dotenv import load_dotenv

import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# get variables from .env file 
load_dotenv()
project_path = os.getenv('PROJECT_PATH')

# add path to load own functions from .py files in Scrips folder
sys.path.insert(0, project_path + '\Scripts')

# from plotting import *
from preprocessing import *


In [3]:
# colors
COLOR_1 = 'steelblue'
RED_COLORS = ['#e3868d', '#E42A38', '#8a010b']  # bright, middle, dark

# 2. Get data

[Data description](data/data_description.md)

In [4]:
# import csv data to pandas dataframes
df_client = pd.read_csv('../data/files/client_train.csv', parse_dates=['creation_date'], dayfirst=True, low_memory=False)
df_invoice = pd.read_csv('../data/files/invoice_train.csv', parse_dates=['invoice_date'], dayfirst=False, low_memory=False)

# Dimensions of dfs
print(f'The client data frame has {df_client.shape[0]} rows and {df_client.shape[1]} columns.')
print(f'The invoice data frame has {df_invoice.shape[0]} rows and {df_invoice.shape[1]} columns.')

The client data frame has 135493 rows and 6 columns.
The invoice data frame has 4476749 rows and 16 columns.


In [5]:
df_client.head(2)

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,1994-12-31,0.0
1,69,train_Client_1,11,107,2002-05-29,0.0


In [6]:
df_invoice.head(2)

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC


# 3. Data cleaning

## Renaming

In [7]:
# rename columns
df_client.rename({'disrict': 'district',
                  'client_catg': 'client_category'
                  }, axis=1, inplace=True)

df_invoice.rename({'counter_statue': 'counter_status',
                   'reading_remarque': 'remark',
                   'counter_coefficient': 'counter_coeff',                   
                   }, axis=1, inplace=True)

# rename 'consommation' features 1 to 4 to consumption
for i in range(1,5):
   df_invoice.rename({f'consommation_level_{i}': f'consumption_lvl_{i}'}, axis=1, inplace=True)
                      
# remove client_id prefix from observations in client_id column
if isinstance(df_invoice['client_id'][0], str):
   df_invoice['client_id'] = df_invoice.client_id.str.removeprefix('train_Client_').astype(int)
   df_client['client_id'] = df_client.client_id.str.removeprefix('train_Client_').astype(int)

df_client.head(2)

,district,client_id,client_category,region,creation_date,target
0,60,0,11,101,1994-12-31,0.0
1,69,1,11,107,2002-05-29,0.0


## Missing values

In [8]:
# check for missing values
na_client = df_client.isna().sum().sum()
na_invoice = df_invoice.isna().sum().sum()
print(f'There are {na_client} missing values in the client data and {na_invoice} missing values in the invoice data.')

There are 0 missing values in the client data and 0 missing values in the invoice data.


## Duplicates

In [9]:
# check for duplicated values
dupes_client = df_client.duplicated().sum()
dupes_invoice = df_invoice.duplicated().sum()
print(f'There are {dupes_client} duplicated values in the client data and {dupes_invoice} duplicated values in the invoice data.')


There are 0 duplicated values in the client data and 11 duplicated values in the invoice data.


In [10]:
# remove duplicated values and check again
df_invoice.drop_duplicates(inplace=True)
df_invoice.duplicated().sum()

0

## Data type conversions

Check the data type (numeric, categorical) and the format (str, int, float) of the values in each column and do conversions where necessary. <br><br>

**Workflow**

---
1.  Check data type of each column with `.info()`
2.  Check number of unique values for each column with `.nunique()`
    - A small number of unique values (usually) indicates a categorical variable
    - A large number of unique values (usually) indicates a numerical variable
    - To be sure look at the unique values
3. Do data type conversion using the function `convert_column_type` from `preprocessing.py` in Scripts directory
    - first convert column data types to int/str
    - then convert column data to categorical if necessary
* Converting a string variable to a categorical variable will save some memory


### Client data

In [11]:
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   district         135493 non-null  int64         
 1   client_id        135493 non-null  int32         
 2   client_category  135493 non-null  int64         
 3   region           135493 non-null  int64         
 4   creation_date    135493 non-null  datetime64[ns]
 5   target           135493 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int32(1), int64(3)
memory usage: 5.7 MB


In [12]:
# getting unique values for each column to get an idea about numerical/categorical features
df_client.nunique()

district                4
client_id          135493
client_category         3
region                 25
creation_date        8088
target                  2
dtype: int64

All features (except the creation_date) will be converted to categorical type.

In [13]:
# step 1: convert to str
cols_to_str = ['district', 'client_category', 'region', 'client_id']
df_client = convert_column_type(df_client, cols_to_str, str)

# step 1: convert to int
df_client = convert_column_type(df_client, 'target', int)

# step 2: convert to category
df_client = convert_column_type(df_client, cols_to_str, 'category') # for percentage calculations 
df_client = convert_column_type(df_client, 'target', 'category')

# check conversions
df_client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135493 entries, 0 to 135492
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   district         135493 non-null  category      
 1   client_id        135493 non-null  category      
 2   client_category  135493 non-null  category      
 3   region           135493 non-null  category      
 4   creation_date    135493 non-null  datetime64[ns]
 5   target           135493 non-null  category      
dtypes: category(5), datetime64[ns](1)
memory usage: 7.1 MB


### Invoice data

In [ ]:
df_invoice.info()

In [ ]:
# getting unique values for each column to get an idea about numerical/categorical features
df_invoice.nunique()

- **Convert to str and category**: <br>['client_id', 'tarif_type', 'counter_status', 'counter_code', 
               'remark', 'counter_coeff', 'old_index', 'new_index', 'counter_type']
- **To be decided**: <br>months_number, counter_number

In [14]:
# step 1: convert to str
cols_to_str = ['client_id', 'tarif_type', 'counter_status', 'counter_code', 
               'remark', 'counter_coeff', 'old_index', 'new_index', 'counter_type']
df_invoice = convert_column_type(df_invoice, cols_to_str, str)

# step 2: convert to category
df_invoice = convert_column_type(df_invoice, cols_to_str, 'category') # for percentage calculations 

df_invoice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4476738 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column             Dtype         
---  ------             -----         
 0   client_id          category      
 1   invoice_date       datetime64[ns]
 2   tarif_type         category      
 3   counter_number     int64         
 4   counter_status     category      
 5   counter_code       category      
 6   remark             category      
 7   counter_coeff      category      
 8   consumption_lvl_1  int64         
 9   consumption_lvl_2  int64         
 10  consumption_lvl_3  int64         
 11  consumption_lvl_4  int64         
 12  old_index          category      
 13  new_index          category      
 14  months_number      int64         
 15  counter_type       category      
dtypes: category(9), datetime64[ns](1), int64(6)
memory usage: 365.6 MB


# 4. Exploratory data analysis (EDA)